In [1]:
from pynq import Overlay
from pynq import MMIO

ol = Overlay("openht.bit")

for item in ol.ip_dict:
    print(item)

baseaddr = int(ol.ip_dict['openht_wrapper_0']['parameters']['C_BASEADDR'], 16)
baseaddr_rx = int(ol.ip_dict['rx_driver/rx_fifo2apb_0']['parameters']['C_BASEADDR'], 16)
baseaddr_tx = int(ol.ip_dict['tx_driver/tx_fifo2apb_0']['parameters']['C_BASEADDR'], 16)

def addr(ip, offset):
    return (ip << 13) + offset*2 

openht_mm = MMIO(baseaddr, 65536)
tx_mm = MMIO(baseaddr_tx, 65536)
rx_mm = MMIO(baseaddr_rx, 65536)

axi_gpio_0
openht_wrapper_0
rx_driver/rx_fifo2apb_0
tx_driver/tx_fifo2apb_0
cpu0


In [ ]:
print(openht_mm.read(addr(3, 8)))

openht_mm.write(addr(2, 4), 1)
openht_mm.write(addr(2, 6), 1)

openht_mm.write(addr(4, 4), 1)
openht_mm.write(addr(4, 6), 1)

openht_mm.write(addr(5, 4), 1)
openht_mm.write(addr(5, 6), 1)

openht_mm.write(addr(6, 4), 1)
openht_mm.write(addr(6, 6), 1)


In [5]:
import socket
import numpy as np

UDP_IP = "0.0.0.0"
TX_UDP_PORT = 10000
TX_RESP_UDP_PORT = 10001

sock = socket.socket(socket.AF_INET, # Internet
                         socket.SOCK_DGRAM) # UDP
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.bind((UDP_IP, TX_UDP_PORT))

def get_data_from_gr(sock):
    while True:
        data, udp_addr = sock.recvfrom(1500)
        float_data = np.frombuffer(data, dtype=np.single) * 0x7FFF
        tx_result = []
        for i, item in enumerate(float_data):
            status = openht_mm.read(addr(0, 10))
            openht_mm.write(addr(0, 8), int(item))

            tx_count = tx_mm.read(0)
            for read in range(tx_count):
                tx_data = tx_mm.read(4)
                tx_data_i = tx_data >> 16
                tx_data_q = tx_data & 0xFFFF

                tx_result.append(tx_data_i if tx_data_i < 32768 else tx_data_i - 65536)
                tx_result.append(tx_data_q if tx_data_q < 32768 else tx_data_q - 65536)

        tx_response = np.asarray(tx_result, dtype=np.single) / 0x8000
        sent = sock.sendto(tx_response, (udp_addr[0], TX_RESP_UDP_PORT))

get_data_from_gr(sock)


KeyboardInterrupt

